# TMDB Box Office Prediction Challenge,
## in this kernel,We're gonna work with a database containing different information about film industries. 
### Could we create a model that helps estimate how to improve movies in order to generate better revenue?

***

## 1. ** loading relevant modules **

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import ast

%matplotlib inline 
#plotting directly without requering the plot()

import warnings
warnings.filterwarnings(action="ignore")

pd.set_option('display.max_columns', 500) #fixing the number of rows and columns to be displayed
pd.set_option('display.max_rows', 500)

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

* LOADING OUR TRAINING SET AND TESTING SET and preparing the dictionary data in our dataframes

* code from this kernel [here](https://www.kaggle.com/gravix/gradient-in-a-box)

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

#### this code is gonna change the dict_columns list below, replacing the NAN values for {}
** ast.literal_eval: **  Safely evaluate an expression node or a string containing a Python literal or container display. 

In [ ]:
dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df
        
train = text_to_dict(train)
test = text_to_dict(test)

In [ ]:
print(train.shape, test.shape) #looking at the shape of our dataframes

* #### By looking at the size of each set, what one might say is that as the test dataset is relativaly large in comparison with our training set, we must try to do our best to not overfit and generalize our model the best

In [ ]:
train.head() #taking a look at the first entries

* some columns have dictionaries inside of it, such **as belongs to a collection**
* before any sort of data cleaning, let's merge the two datasets together to avoid code repetition

In [ ]:
##Merging the train and test dataset in order to have more data to train our model.

train['source']='train' #creating a label for the training and testing set
test['source']='test'

data = pd.concat([train, test],ignore_index=True)
print (train.shape, test.shape, data.shape) #printing the shape

In [ ]:
print("Belongs to a collection or not:\n%s" % (data['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0).value_counts()))

* the vast majority of the data does not belong to a collection, so we might as well create a boolean column **has_collection** that gives us this information
* moreover, we don't need the dictionary, but only the name of the collection is enough

## *** Feature Engineering and Data Cleaning ***

#### One hot coding, creating boolean columns depending on the weight of values 
* in order to make it easier, it's gonna be created a function to read the dictionaries and create the boolean columns for each most common values

In [ ]:
## Function that receives the dataset, reads the dictionary and create a new boolean column for each top 20 common value
def dict_one_hot_code(dataset,name_column, name_new_column):
    list_of_values = list(dataset[name_column].apply(lambda x: [i['name'] for i in x] if x != {} else []).values) #create a list of each occurence of a given name
    Counter([i for j in list_of_values for i in j]).most_common(20)
    top_list_values = [m[0] for m in Counter([i for j in list_of_values for i in j]).most_common(20)]
    for g in top_list_values: 
        dataset[name_new_column + "_" + g] = dataset[name_column].apply(lambda x: 1 if g in str(x) else 0)
        
def counting_number_of_values_dict(dataset, name_column):
    return dataset[name_column].apply(lambda x: len(x) if x!= {} else 0)

In [ ]:
data['collection_name'] = data['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else "None")
data['is_part_of_collection'] = data['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0) 
#feature engineering a new column that has the len of the belongs_to_collection columns, by doing so, you attribute a weight to each particular movie, or zero if it doesn't belong to any

data.drop('belongs_to_collection', axis = 1, inplace=True)

In [ ]:
data.head() #checking the changes 

**Now let's take a look at other dictionaries we have in this dataset**

In [ ]:
#count the number of genres
data['genres'].apply(lambda x: x[0]['name'] if x != {} else 0).value_counts() #this counts for us the number of dicts containing the genre in our dataset and count the number of occurences

### As it was done for collections, only the name of the genre is importante, and popularity of this given genre. Notice that we have 23 movies that the genre have not been listed(value 0)

In [ ]:
data['genre'] = data['genres'].apply(lambda x: x[0]['name'] if x != {} else "None") # exactly the same code used for the collection belongs.
data.drop('genres', axis=1, inplace=True)

* next, take a look at cast

**looking at the first entry to have an idea of how the data is**

In [ ]:
data['cast'][0]

In [ ]:
data['cast'][0][0]

** that's a lot of data, only the relevant characters would be important **

The code below has been take form this kernel: [here](https://www.kaggle.com/artgor/eda-feature-engineering-and-model-interpretation)

In [ ]:
list_of_cast_names = list(data['cast'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values) #create a list of each occurence of a given name
Counter([i for j in list_of_cast_names for i in j]).most_common(20) #count every instance of each actor and sum them all to return back the 20 most common instances

In [ ]:
#and let's also take a look at the total number of cast member for each movie
data['cast'].apply(lambda x: len(x) if x != {} else 0).value_counts()

** 15 person have been casted in 525 different movies **

Gonna create a column with the number of cast members for each movie.

In [ ]:
data['number_cast'] = counting_number_of_values_dict(data,'cast')

In [ ]:
data.head()

**And again, next step I got the idea and code from this kernel ** [here](https://www.kaggle.com/artgor/eda-feature-engineering-and-model-interpretation)

##### I am not gonna get the gender out of the cast column, for our model, I believe what is the most important is the actor's name and kind of character

In [ ]:
list_of_cast_characters = list(data['cast'].apply(lambda x: [i['character'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_cast_characters for i in j]).most_common(15)

In [ ]:
top_cast_names = [m[0] for m in Counter([i for j in list_of_cast_names for i in j]).most_common(15)]
for g in top_cast_names: 
    data['cast_name_' + g] = data['cast'].apply(lambda x: 1 if g in str(x) else 0) #creating a boolean for the top 15 most common actors...

In [ ]:
data.head()

** last step has increase the number of features significantly, but not drammatically, and the models are going to find relationships way faster with boolean columns like so **

In [ ]:
data.shape

* same thing will be done with top 15 most common characters

In [ ]:
top_cast_characters = [m[0] for m in Counter([i for j in list_of_cast_characters for i in j]).most_common(15)]
for g in top_cast_characters:
    data['cast_character_' + g] = data['cast'].apply(lambda x: 1 if g in str(x) else 0)

**Finally we can drop the cast column and next, we are going to take a look at the crew column**

In [ ]:
data.drop('cast', axis=1, inplace=True)

Similar to what was done to the cast column, it's gonna be done to the crew column


* crew total number for each movie

In [ ]:
counting_number_of_values_dict(data,'crew').value_counts()

gonna create a crew_total

In [ ]:
data['crew_total'] = counting_number_of_values_dict(data, 'crew')

##### Again, using the same code that has been used for the cast, let's take a look at the top 15 most common crew members

In [ ]:
dict_one_hot_code(data,name_column='crew',name_new_column='crew_name')

** IMO, it's enough the name of crew members and tota crew members as it has been created, next, I am gonna create booean columns for each of top 15 crew members and drop the crew column **

In [ ]:
data.drop('crew', axis=1, inplace=True)

#### 3 more dictionaries to extract relevant information for our models

In [ ]:
data.shape

** Checking the first entry for keywords, we have different keywords for each movie, similar to what has been done for the other dictionaries, I am gonna keep only the most common keywords**
* eg: Hot Tube is not a recurrent keyword, let's check

In [ ]:
data.head()

In [ ]:
data['Keywords'][0]

In [ ]:
counting_number_of_values_dict(data,'Keywords').value_counts()

** following the same logic for other dictionaries, gonna create a column with the total number of keywords for each movie **

In [ ]:
data['keywords_num'] = counting_number_of_values_dict(data,'Keywords')

In [ ]:
list_of_keywords = list(data['Keywords'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)

** creating a column with all the keywords presents in the dictionary **

In [ ]:
data['all_Keywords'] = data['Keywords'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')

** Same code as it has been used before for the other dicts, it's getting the list of keywords to be used with the counter method to get the 30 most common, then create a boolean column for each keyword**

In [ ]:
top_keywords = [m[0] for m in Counter([i for j in list_of_keywords for i in j]).most_common(30)]
for g in top_keywords:
    data['keyword_' + g] = data['all_Keywords'].apply(lambda x: 1 if g in x else 0)

* Finally, dropping the **Keywords** and **All Keywords** column

In [ ]:
data.drop(['Keywords','all_Keywords'], axis=1, inplace=True)

* Now on the spoken languages column

In [ ]:
counting_number_of_values_dict(data,'spoken_languages').value_counts()

In [ ]:
languages = list(data['spoken_languages'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
Counter([i for j in languages for i in j]).most_common(15)

* Creating a column for the number of the languages spoken for each movie.

In [ ]:
data['num_languages'] = counting_number_of_values_dict(data,'spoken_languages')
data['all_languages'] = data['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
top_languages = [m[0] for m in Counter([i for j in languages for i in j]).most_common(30)]
for g in top_languages:
    data['language_' + g] = data['all_languages'].apply(lambda x: 1 if g in x else 0)

In [ ]:
data.drop(['spoken_languages', 'all_languages'], axis=1, inplace=True)

In [ ]:
data.shape #number of features has increased significantly

* Now on to the production_countries

In [ ]:
counting_number_of_values_dict(data,'production_countries').value_counts()

* checking the total number of countries participating in each movies 

* creating a column with the number of contries participating in the movie

In [ ]:
data['num_countries'] = counting_number_of_values_dict(data, 'production_countries')
dict_one_hot_code(data,name_column= 'production_countries', name_new_column='production_country')


In [ ]:
data.head()

In [ ]:
data.drop('production_countries', axis=1, inplace=True) #and finally, dropping the dictionary...

** finally, on to the last dictionary, production companies. **
* I am not sure whether the number of production companies a movie has gives it a better chance of success, gotta check this later..

In [ ]:
counting_number_of_values_dict(data,'production_companies').value_counts()

In [ ]:
data['prod_companies_total_number'] = counting_number_of_values_dict(data,'production_companies')

In [ ]:
dict_one_hot_code(data,'production_companies','all_production_companies')

In [ ]:
data.drop('production_companies', axis=1, inplace=True)

In [ ]:
data.shape

In [ ]:
data.head()

* taking a look at the homepage column

In [ ]:
data['homepage'].isnull().value_counts() 

** instead of using the name of the homepage, it would be better to create a column has_homepage, indicating whether or not a given movie has a homepage or not **

In [ ]:
data['has_homepage'] = 0 #setting all the values to false as default
data.loc[data['homepage'].isnull() == False, 'has_homepage'] = 1
data.drop('homepage', axis=1, inplace=True)

***

## Now that data has been prepared, a brief data analysis in order to derive some insights out of our dataset relate to the revenue, our target

***

* taking a look at the distribution of different genres and the density plot of revenue.

In [ ]:
# Set the style of plots
plt.style.use('fivethirtyeight')

fig, ax = plt.subplots(2,2, figsize = (16, 6))

plt.subplot(1, 2, 1)
sns.countplot(y=data['genre'])
plt.title('Distribution of genres')
plt.subplot(1, 2, 2)
sns.kdeplot(data['revenue'])
plt.title('Distribution of revenues(nomal scale)')

** As one could have expected, distribution of revenues is skewed, and the most representative genre is comedy**
* it's gonna be used the log 1 for our revenue, so we are gonna have a better idea of the distribution

In [ ]:
data['revenue_log'] = np.log1p(data['revenue']) #changing it to log1
sns.kdeplot(data['revenue_log'])
plt.title('Distribution of revenues in log scale')

** that looks more promissing, despite having a long tail to the right, it's better distributed now, it's gonna help our analysis for now on...**
* we are gonna gonna use log1 for budget as well, and we are gonna see how budget is ditributed and wha's the relationship with revenue 

In [ ]:
fig, ax = plt.subplots(figsize = (8, 10))
data['budget_log'] = np.log1p(data['budget'])

plt.subplot(2,1,1)
plt.scatter(x = data['budget'], y=data['revenue']);plt.ylabel('Revenue'); plt.xlabel('budget')
plt.title('distribution of revenue X budget')
plt.subplot(2,1,2)
plt.scatter(x = data['budget_log'], y=data['revenue']);plt.ylabel('Revenue');plt.xlabel('budget')
plt.title('distribution of revenue X budget(log scale)')
plt.tight_layout()


In [ ]:
sns.kdeplot(data['budget_log'])
plt.title('Density distribution of budget(log scale)')
plt.tight_layout()

## we have a really strong, linear and positive, relationship between revenue and budget, and the density distribution shows two different aggregates, what does that mean? an assymetry in the distribution of budgets? that's what our intuition tell us, isn't it?

*** 


* is there a relationship between having a homepage and reveue? And orginal language? how does it influences the revenue?

In [ ]:
sns.catplot(x='has_homepage', y='revenue', data=data, aspect=1.5); plt.title('Has homepage X Revenue')
sns.catplot(x='is_part_of_collection', y='revenue', data=data, aspect=1.5); plt.title('Part of colection X Revenue')

In [ ]:
data_language = pd.DataFrame(data.groupby('original_language')["revenue"].sum().sort_values(ascending=False).head(10))

In [ ]:
data_language #showing only the first 10

** that's interesting, isn't? having a website have a positive correlation to revenue **
** it's also quite informative the fact that most of our data is populated with english movies, gonna create a new category, is_english_original_language, and see how it correlates to revenue **



In [ ]:
data['is_english_original_language'] = 0 #setting all the values to false as default
data.loc[data['original_language'] == 'en', 'is_english_original_language'] = 1

In [ ]:
sns.catplot(x='is_english_original_language', y='revenue', data=data)
plt.title('Revenue in relationship to english as original language')

**As be can be seen, english as an original language is correlated to a higher revenue. I am gonna drop the column original language now**

In [ ]:
data.drop('original_language', axis=1, inplace=True)

***

#### now, The popularity rating of a particular title on IMDb indicates how much that title's page has been visited in the current week on IMDb in it's area of interest. 
* plotting how popularity impacts revenue

In [ ]:
plt.scatter(x = data['popularity'], y=data['revenue'])
plt.ylabel('revenue')
plt.xlabel('popularity')
plt.title('correlation of popularity X revenue')

#### popularity has a positive correlation to revenue, as it could have been guessed.

##### release_date has some problems that need to be fixed, let's address this now

In [ ]:
data[['release_month','release_day','release_year']]=data['release_date'].str.split('/',expand=True).replace(np.nan, -1).astype(int) #getting the month year and day using the string split function and the / as a delimiter; eg: 5/25/2015 -> month 5/ day 25 / year 2015
data.loc[ (data['release_year'] <= 19) & (data['release_year'] < 100), "release_year"] += 2000 ## some rows have 4 digits for the year instead of 2, so the release year < 100 and > 100 is checking that
data.loc[ (data['release_year'] > 19)  & (data['release_year'] < 100), "release_year"] += 1900

releaseDate = pd.to_datetime(data['release_date']) #using the pandas to_datetime function to format the data, get a Series,  and store it in a variable that is gonna be used later to get the day of week and quarter
data['release_dayofweek'] = releaseDate.dt.dayofweek
data['release_quarter'] = releaseDate.dt.quarter

In [ ]:
plt.figure(figsize=(20, 8))
plt.scatter(x='release_year', y='revenue', data=data); plt.title('Revenue X year released');

#### Clearly, as could've been expected, revenue has been increasing in time...

* Continuing checking what's the distribution for days of the week and days of the month.

In [ ]:
plt.figure(figsize=(15, 8))
sns.countplot(data['release_dayofweek']); plt.title('Days of the week, from 0 - sunday,  to 6 - saturday')

In [ ]:
## -1 is the nan values in the original data, that has been replaced by -1
plt.figure(figsize=(15, 8))
sns.countplot(data['release_month']); plt.title('Distribution of movies by month')

In [ ]:
plt.figure(figsize=(15, 8))
sns.countplot(data['release_day']); plt.title('Distribution of movies by days in a month')

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 3, 1)
plt.hist(data['runtime'].fillna(0) / 60, bins=40) #filling the null values with 0
plt.title('Distribution of length of film in hours');
plt.subplot(1, 3, 2)
plt.scatter(data['runtime'].fillna(0), data['revenue'])
plt.title('runtime vs revenue');
plt.subplot(1, 3, 3)
plt.scatter(data['runtime'].fillna(0), data['popularity'])
plt.title('runtime vs popularity');

** There is a positive relationship between the length of the movie and revenue, there's also a positive relationship between length and popularity but it is less pronounced**

## using all the features previously created, such as number of crew members, number of cast, number of companies and so on

In [ ]:
plt.figure(figsize=(16, 14))
plt.subplot(3, 3, 1)
plt.scatter(data['number_cast'], data['revenue'])
plt.title('Revenue X number of cast members')
plt.subplot(3, 3, 2)
plt.scatter(data['crew_total'], data['revenue'])
plt.title('Revenue X number of crew members')
plt.subplot(3, 3, 3)
plt.scatter(data['num_languages'], data['revenue'])
plt.title('Revenue X number of languages')
plt.tight_layout(1.5)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(2, 2, 1)
plt.scatter(data['num_countries'], data['revenue'])
plt.title('Revenue X number of countries')
plt.subplot(2, 2, 2)
plt.scatter(data['keywords_num'], data['revenue'])
plt.title('Revenue X number of keywords')

** plenty of information in those plots: **
* number of keywords from 0 - 40 is the strongly present in this data, whereas we have two movies witch a low revenue and a high number of keywords
* number of countries have also a negative correlation the more countries we have
* the same thing goes for number of languages

***

### We have some features in this dataset that has descriptive information, those columns have valuable information that may help our models. I am gonna try simply getting the length of those features and see whether it has a positive relationship with our target and to what extent

In [ ]:
data['title_length'] = data['original_title'].apply(lambda x: len(x) if x != '' else 0)

In [ ]:
plt.figure(figsize=(20, 8))
plt.scatter(data['title_length'], data['revenue']);plt.title('Revenue X title lenght')

** Like pretty much all the other features where we have sums, higher revenues are correlated with titles within a normal range, less than 30**
* This is for the original title, now, what about the actual title, is there gonna be a difference?
#TODO

***
Wrapping up our EDA, it might be informative to take a look at the linear correlation among our features.

In [ ]:
corr = data.corr()
corr['revenue'].sort_values(ascending=False).head(20) #the top 20 features positively correlated to our target

In [ ]:
corr['revenue'].sort_values(ascending=False).tail(20) #negatively correlated to our target

## Models
##### Start modeling  

In [ ]:
data.isnull().sum().sort_values(ascending=False)

#### missing values won't be a problem, tagline is gonna be drop, overview as well... runtime can be filled with the median value.
* dropping some columns that are not really important for our models

In [ ]:
data.drop(['imdb_id', 'poster_path', 'release_date', 'status','budget_log','cast_character_','language_'], axis=1, inplace=True)

In [ ]:
data_dropping_names = data.drop(['original_title','overview','tagline','title'], axis=1)

train = data_dropping_names[data_dropping_names['source'] == 'train'].copy()
test = data_dropping_names[data_dropping_names['source'] == 'test'].copy()

train_labels = train['revenue_log'] #creating labels, our Y_train, gonna use the log as it works better for skewed data

train.drop(['id', 'revenue', 'source', 'revenue_log'], axis=1, inplace=True) # dropping the target and id

test_final = test.drop(['id', 'source','revenue','revenue_log'], axis=1) #this is the final test, the dataset give for our prediction, notice that I am dropping the revenue column here that has been created when we merged the two datasets together

In [ ]:
print(train.shape, train_labels.shape,test_final.shape)

In [ ]:
train.isnull().sum()

* Creating a simple numerical pipeline to impute and scale the data

In [ ]:
#this pipeline is gonna be use for numerical atributes and standard scaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,RobustScaler, MinMaxScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('robust_scaler', RobustScaler()),
        #('minmax_scaler', MinMaxScaler())
    ])

## Cool, our rather simple pipeline has been create, now I am gonna implement a function that is an "all-in-one" function, which has been implemented in this kernel [here](https://www.kaggle.com/artgor/eda-feature-engineering-and-model-interpretation)
* this function receives different models, parameters, and the Kfold that is gonna be used to train and validate the model using different folds.

In [ ]:
#let's create this function to make it easier and clean to fit the model and use the cross_val_score and obtain results
import time #implementing in this function the time spent on training the model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import  KFold
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb
import eli5
import gc

n_fold = 10 #number of folds that our function is gonna use and split the training set accordingly
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

def train_model(X, X_test, y, params=None, folds=folds, model_type='lgb', plot_feature_importance=False, model=None):

    prediction = np.zeros(X_test.shape[0]) #initializing the prediction matrix with zeros, with the number of training examples in X_test
    scores = [] #this list is gonna be used to store all the scores across different folds
    feature_importance = pd.DataFrame() #initializing this dataframe, it's gonna be used to plot the features importance.
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        if model_type == 'sklearn': #if the model type is sklearn then
            X_train, X_valid = X[train_index], X[valid_index]
        else:
            X_train, X_valid = X.values[train_index], X.values[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if model_type == 'lgb': 
            model = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
                    verbose=1000, early_stopping_rounds=200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test.values), ntree_limit=model.best_ntree_limit)

        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = mean_squared_error(y_valid, y_pred_valid)
            
            y_pred = model.predict(X_test)
            
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric='RMSE', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        scores.append(mean_squared_error(y_valid, y_pred_valid) ** 0.5)
        
        prediction += y_pred #summing all the prediction which is gonna later be divided by the number of folds   
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold # all the predictions divided by the number of folds(getting the average value)
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    # Clean up memory
    gc.enable()
    del model, y_pred_valid, X_test,X_train,X_valid, y_pred, y_train
    gc.collect()

    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return scores, prediction, feature_importance 
        return scores, prediction
    else:
        return scores, prediction
    

## the function implemented above creates a validation set for each Kfold, depending on the number of splits you want. I set it to 10 splits.

* creating also a training set with dummies, as well as align it with the testing set to avoid different columns

In [ ]:
train_dummies = pd.get_dummies(train)
test_dummies = pd.get_dummies(test)
train_dummies, test_dummies = train_dummies.align(test_dummies, axis=1, join='inner')

In [ ]:
print(train_dummies.shape, test_dummies.shape)

* Starting testing our models, first is gonna be LGB

In [ ]:
########## LGB ########
params = {
          'num_leaves': 30,
         'min_data_in_leaf': 20,
         'objective': 'regression',
         'max_depth': 6,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2,
}
score_lgb, prediction_lgb, _ = train_model(train_dummies, test_dummies, train_labels, params=params, model_type='lgb', plot_feature_importance=True)

### Now, plotting our RMSE

In [ ]:
score_lgb.sort(reverse=True)
dictvalues = {'RMSE_lgb': score_lgb}

In [ ]:
plt.plot(dictvalues['RMSE_lgb'], label = 'RMSE_lgb')
plt.legend()

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['revenue'] = np.expm1(prediction_lgb)
sub.to_csv("lgb_model.csv", index=False)

In [ ]:
xgb_params = {'eta': 0.01,
              'objective': 'reg:linear',
              'max_depth': 7,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              'eval_metric': 'rmse',
              'seed': 11,
              'silent': True}
score_xgb, prediction_xgb = train_model(train_dummies, test_dummies, train_labels, params=xgb_params, model_type='xgb', plot_feature_importance=True)

In [ ]:
score_xgb.sort(reverse=True)
dictvalues.update({'RMSE_XGB': score_xgb})

In [ ]:
plt.plot(dictvalues['RMSE_lgb'], label = 'RMSE_lgb')
plt.plot(dictvalues['RMSE_XGB'], label = 'RMSE_XGB')
plt.legend()

In [ ]:
sub['revenue'] = np.expm1(prediction_xgb)
sub.to_csv("XGB_model.csv", index=False)

Quite similar both models, don't you think? In fact, in small datasets as this one, XGB tends to outpeform lgb...
* next, catboost...

In [ ]:
cat_params = {'learning_rate': 0.002,
              'depth': 5,
              'l2_leaf_reg': 10,
              'colsample_bylevel': 0.8,
              'bagging_temperature': 0.2,
              'od_type': 'Iter',
              'od_wait': 100,
              'random_seed': 11,
              'allow_writing_files': False}
score_cat, prediction_cat = train_model(train_dummies, test_dummies, train_labels, params=cat_params, model_type='cat', plot_feature_importance=True)

In [ ]:
score_cat.sort(reverse=True)
dictvalues.update({'RMSE_CAT': score_cat})
plt.plot(dictvalues['RMSE_lgb'], label = 'RMSE_lgb')
plt.plot(dictvalues['RMSE_XGB'], label = 'RMSE_XGB')
plt.plot(dictvalues['RMSE_CAT'], label = 'RMSE_CAT')
plt.legend()

In [ ]:
sub['revenue'] = np.expm1(prediction_cat)
sub.to_csv("cat_model.csv", index=False)

* gonna combine all the prediction and divide by 3, honestly, I don't like this approach, this is only for kaggle....

In [ ]:
sub['revenue'] = np.expm1((prediction_lgb + prediction_xgb + prediction_cat) / 3)
sub.to_csv("combined.csv", index=False)

***
# This next part, I am using keras and building a simple neural net regressor. I tried to use it with a scikit learn wrapper but I couldn't make it work, for some reason, I am gonna try it again later. This won't get a good score, but it's for educational purposes. It's fun! 
***

In [ ]:
## loading relevant models for this part of the notebook
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.layers import Dense, Activation, Dropout, Input, concatenate,BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [ ]:
########### creating a function ir order to make easier to test different models ###############

def build_model(input_shape,n_hidden=10, n_neurons=30, optimizer = SGD(3e-3,momentum=0.9)): # the function has the stochastic gradient descent with momemntum as the default, you might wanna use it and see the results
    model = Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden): # fixed number of neurons for each hidden layer
        BatchNormalization() #calling batch normalization before the activation...
        model.add(Dense(n_neurons, activation="relu",
                           kernel_regularizer=l2(0.01), **options)) #using the relu activation and L2 regularization, I couldn't make this NN works better and the model didn't generalize well
        options = {}
    BatchNormalization() #adding BN before the output layer
    model.add(Dense(1, **options)) 
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

In [ ]:
train_dummies_prepared = num_pipeline.fit_transform(train_dummies)

In [ ]:
model_NN = build_model(input_shape= train_dummies_prepared.shape[1:], n_hidden=6,optimizer=Adam(3e-5)) #gonna use 6 hidden layers and Adam 

In [ ]:
model_history = model_NN.fit(train_dummies_prepared, train_labels, epochs=2500, batch_size=5, validation_split=0.1,callbacks=[EarlyStopping(patience=100)])

In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 50) # set the vertical range to [0-1]

### I would love to improve this neural network, any sort of feedback is welcome...